In [1]:
import test_config as cfg
import numpy as np
import itertools, functools

%matplotlib inline
import matplotlib.pyplot as plt
import sys, os, os.path, time, datetime
import pickle, io, json

import skimage, skimage.io, skimage.transform, skimage.filters
import sklearn, sklearn.metrics

import importlib
sys.path.append('../src/')
import modutils
import word_processing as wp
import htr_model as hm
import tqdm
import tensorflow as tf
import editdistance

In [12]:
importlib.reload(hm)

<module 'htr_model' from '../src\\htr_model.py'>

In [2]:
class BaseTransformer:
    def __init__(self):
        pass
    def transform(self, x):
        return x

    
class SequentialTransformer:
    def __init__(self, *args):
        self.stages_ = args
        
    def transform(self, x):
        res = x
        for s in self.stages_:
            res = s.transform(res)
        return res
    
class LoadImageTransformer(BaseTransformer):
    def __init__(self, path):
        self.path_ = path
        
    def transform(self, x):
        if type(x) != str:
            raise Exception("LoadImageTransformer: expects filename as argument!")
        return skimage.io.imread(os.path.join(self.path_, x), as_grey=True)
    
class ConvertFloatTransformer(BaseTransformer):
    def __init__(self, min_value = 0.0, max_value = 1.0):
        self.min_ = min_value
        self.max_ = max_value
        
    def transform(self, x):
        if x.dtype in (np.float, np.float64, np.float32):
            return x
        if x.dtype == np.uint8:
            return (x / 255.0) * (self.max_ - self.min_) + self.min_
        if x.dtype == np.uint16:
            return (x / 65535.0) * (self.max_ - self.min_) + self.min_
        raise Exception("ConvertFloatTransformer: unexpected argument type!")
    
class RandomStretchTransformer(BaseTransformer):
    def __init__(self, min_scale = 0.66, max_scale = 1.5, fill_value=1.0):
        self.max_ = max_scale
        self.min_ = min_scale
        self.fill_ = fill_value
        
    def transform(self, x):
        f = np.random.uniform(self.min_, self.max_)
        return skimage.transform.rescale(x, (1.0, f), mode='constant', cval=self.fill_)
    
class TransposeTransformer(BaseTransformer):
    def __init__(self):
        pass
    def transform(self, x):
        return np.transpose(x)
    
class FitSizeTransformer(BaseTransformer):
    def __init__(self, width, height, fill_value=1.0):
        self.w_ = width
        self.h_ = height
        self.fill_ = fill_value
        self.template_ = np.ones((self.h_, self.w_)) * self.fill_
        
    def transform(self, x):
        (h, w) = x.shape
        f = max(w / self.w_, h / self.h_)
        res = self.template_.copy()
        rw = max(min(self.w_, int(w / f)), 1)
        rh = max(min(self.h_, int(h / f)), 1)
        res[0:rh, 0:rw] = skimage.transform.resize(x, (rh, rw), mode='constant', cval=self.fill_)
        return res
    
class StandardizeTransformer(BaseTransformer):
    def __init__(self):
        pass
    
    def transform(self, x):
        m = np.mean(x)
        s = np.std(x)
        if s <= 1e-9:
            return x - m
        return (x - m) / s
    
class TruncateLabelTransform(BaseTransformer):
    def __init__(self, max_cost):
        self.max_cost_ = max_cost
        
    def transform(self, x):
        if type(x) != str:
            raise Exception("TruncateLabelTransform: input expected to be of type string!")
        cost = 0
        for i in range(len(x)):
            flg = (i > 0) and (x[i] == x[i-1])
            cost += 1 + int(flg)
            if cost > max_cost:
                return x[:i]
        return x

In [23]:
def format_timedelta(seconds):
    if seconds < 1e-10:
        return '0s'
    sf = seconds - np.floor(seconds)
    si = int(np.floor(seconds))
    d, s_h = divmod(si, 3600*24)
    h, s_m = divmod(s_h, 3600)
    m, s = divmod(s_m, 60)
    if d > 9:
        return '{}d'.format(d)
    elif d > 0:
        return '{}d {}h'.format(d, h)
    elif h > 9:
        return '{}h'.format(h)
    elif h > 0:
        return '{}h {}m'.format(h, m)
    elif m > 9:
        return '{}m'.format(m)
    elif m > 0:
        return '{}m {}s'.format(m, s)
    elif s > 9:
        return '{}s'.format(s)
    elif s > 0:
        return '{:.1f}s'.format(s + sf)
    else:
        return '{}ms'.format(int(sf*1000))
    
def apply_esmooth(array, factor):
    tmp = np.exp(np.cumsum([factor]*len(array)))
    tmp = tmp / np.sum(tmp)
    return np.sum(tmp * array)
    
def train(model, imgs, labels, batch_size, transform_pipeline=BaseTransformer()):
    num = len(imgs)
    num_batches = num // batch_size
    ids = np.arange(num)
    np.random.shuffle(ids)
    text_template = 'Train batch {}/{}. Loss: {:.2f}. Time: {}. ETA: {}.'
    hist_times = []
    t_start = time.perf_counter()
    sum_loss = 0
    for i in range(num_batches):
        t0 = time.perf_counter()
        batch_ids = ids[(i*batch_size):((i+1)*batch_size)]
        batch_imgs = np.array([transform_pipeline.transform(imgs[j]) for j in batch_ids])
        batch_lbls = np.array([labels[j] for j in batch_ids])
        loss = model.trainBatch(batch_imgs, batch_lbls)
        sum_loss += loss * len(batch_lbls)
        t1 = time.perf_counter()
        hist_times.append(t1-t0)
        t_delta = apply_esmooth(np.array(hist_times)[::-1], -0.5)
        t_eta = t_delta * (num_batches - i - 1)
        print(text_template.format(i+1, num_batches, loss, format_timedelta(t1-t0), format_timedelta(t_eta)))
    return sum_loss / len(labels)
        
def validate(model, imgs, labels, batch_size,
             transform_pipeline=BaseTransformer()):
    n_char_err = 0
    n_char = 0
    n_word_ok = 0
    n_word = 0
    num_batches = len(imgs) // batch_size
    text_template = 'Validation batch {}/{}. Time: {}. ETA: {}.'
    hist_times = []
    t_start = time.perf_counter()
    sum_loss = 0
    for i in range(num_batches):
        t0 = time.perf_counter()
        batch_imgs = np.array([transform_pipeline.transform(x)
                               for x in imgs[(i*batch_size):((i+1)*batch_size)]])
        batch_lbls = np.array(labels[(i*batch_size):((i+1)*batch_size)])
        recognized, loss = model.validBatch(batch_imgs, batch_lbls)
        sum_loss += loss * len(batch_lbls)
        for j in range(len(recognized)):
            n_word_ok += int(batch_lbls[j] == recognized[j])
            n_word += 1
            dist = editdistance.eval(recognized[j], batch_lbls[j])
            n_char_err += dist
            n_char += len(batch_lbls[j])
        t1 = time.perf_counter()
        hist_times.append(t1-t0)
        t_delta = apply_esmooth(np.array(hist_times)[::-1], -0.5)
        t_eta = t_delta * (num_batches - i - 1)
        t_eta = (t1 - t_start) / (i + 1) * (num_batches - i - 1)
        print(text_template.format(i+1, num_batches, format_timedelta(t1-t0), format_timedelta(t_eta)))

    cer = n_char_err / n_char
    wa = n_word_ok / n_word
    print('Validation results: CER: {:.3f}, WA: {:.3f}.'.format(cer, wa))
    return sum_loss/len(labels), cer, wa

def infer(model, imgs, batch_size, transform_pipeline=BaseTransformer()):
    num_batches = len(imgs) // batch_size
    text_template = 'Inference batch {}/{}. Time: {}. ETA: {}.'
    hist_times = []
    t_start = time.perf_counter()
    result = []
    for i in range(num_batches):
        t0 = time.perf_counter()
        batch_imgs = np.array([transform_pipeline.transform(x)
                               for x in imgs[(i*batch_size):((i+1)*batch_size)]])
        result += model.inferBatch(batch_imgs)
        t1 = time.perf_counter()
        hist_times.append(t1-t0)
        t_delta = apply_esmooth(np.array(hist_times)[::-1], -0.5)
        t_eta = t_delta * (num_batches - i - 1)
        t_eta = (t1 - t_start) / (i + 1) * (num_batches - i - 1)
        print(text_template.format(i+1, num_batches, format_timedelta(t1-t0), format_timedelta(t_eta)))

    print('Inference complete.')
    return result

def inferRaw(model, imgs, batch_size, transform_pipeline=BaseTransformer()):
    num_batches = len(imgs) // batch_size
    text_template = 'Inference batch {}/{}. Time: {}. ETA: {}.'
    hist_times = []
    t_start = time.perf_counter()
    result = []
    for i in range(num_batches):
        t0 = time.perf_counter()
        batch_imgs = np.array([transform_pipeline.transform(x)
                               for x in imgs[(i*batch_size):((i+1)*batch_size)]])
        result.append(model.inferRaw(batch_imgs))
        t1 = time.perf_counter()
        hist_times.append(t1-t0)
        t_delta = apply_esmooth(np.array(hist_times)[::-1], -0.5)
        t_eta = t_delta * (num_batches - i - 1)
        t_eta = (t1 - t_start) / (i + 1) * (num_batches - i - 1)
        print(text_template.format(i+1, num_batches, format_timedelta(t1-t0), format_timedelta(t_eta)))

    print('Inference complete.')
    return np.vstack(result)

def run_training(model, train_imgs, train_labels, valid_imgs, valid_labels,
                 batch_size=128, train_pipeline=BaseTransformer(), valid_pipeline=None):
    epoch = 0
    if valid_pipeline is None:
        valid_pipeline = train_pipeline
    text_template = 'Epoch {} complete in {}. T-loss is {:.2f}, V-loss is {:.2f}'
    while True:
        t0 = time.perf_counter()
        epoch += 1
        print('Epoch: {}'.format(epoch))

        tloss = train(model, train_imgs, train_labels, batch_size, transform_pipeline=train_pipeline)
        vloss, cer, wa = validate(model, valid_imgs, valid_labels, batch_size, transform_pipeline=valid_pipeline)
        model.save()
        t1 = time.perf_counter()
        print(text_template.format(epoch, format_timedelta(t1-t0), tloss, vloss))
        
def load_sample(fname):
    path = '.'.join(fname.split('.')[:-1])
    sample = [tuple(y.strip() for y in x.split(' ')) for x in open(fname, 'r').readlines()]
    load_pipeline = SequentialTransformer(LoadImageTransformer(path), ConvertFloatTransformer())
    return [x for x in sample if len(x[1])>0], load_pipeline

        
def prepare_sample(sample, pipeline):
    imgs = [pipeline.transform('{}.png'.format(x)) for (x,_) in sample]
    lbls = [x for (_,x) in sample]
    return imgs, lbls

In [5]:
train_sample, train_load_pipeline = load_sample('D:/Data/HTR/train.txt')
train_imgs, train_lbls = prepare_sample(train_sample, train_load_pipeline)

C:\Anaconda3\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


In [6]:
valid_sample, valid_load_pipeline = load_sample('D:/Data/HTR/valid.txt')
valid_imgs, valid_lbls = prepare_sample(valid_sample, valid_load_pipeline)

In [7]:
transform_pipeline = SequentialTransformer(
    FitSizeTransformer(128, 32),
    TransposeTransformer(),
    StandardizeTransformer())
charlist = sorted(list(functools.reduce(set.union, [set(x) for x in train_lbls])))

In [13]:
model = hm.HTRModel(charlist, img_size=(128, 32),
                cnn_kernels = [5, 5, 3, 3], #[5, 5, 3, 3, 3],#default
                cnn_features = [1, 32, 64, 64, 128], #[1, 32, 64, 128, 128, 256],#default
                cnn_pools = [(2,2), (2,2), (1,2), (1,4)], #[(2,2), (2,2), (1,2), (1,2), (1,2)] #default
                rnn_cells = [128, 128], #default 
                model_path='D:/models/htr-static-128/'
                )

Python: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]; TF: 1.14.0
Starting hot: D:/models/htr-static-128/snapshot-178


In [9]:
tloss, tcer, twa = validate(model, train_imgs, train_lbls, 256, transform_pipeline)

C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Validation batch 1/54. Time: 2.2s. ETA: 1m 56s.
Validation batch 2/54. Time: 1.3s. ETA: 1m 32s.
Validation batch 3/54. Time: 1.4s. ETA: 1m 24s.
Validation batch 4/54. Time: 1.3s. ETA: 1m 18s.
Validation batch 5/54. Time: 1.3s. ETA: 1m 13s.
Validation batch 6/54. Time: 1.2s. ETA: 1m 9s.
Validation batch 7/54. Time: 1.3s. ETA: 1m 7s.
Validation batch 8/54. Time: 1.2s. ETA: 1m 4s.
Validation batch 9/54. Time: 1.3s. ETA: 1m 2s.
Validation batch 10/54. Time: 1.2s. ETA: 1m 0s.
Validation batch 11/54. Time: 1.4s. ETA: 59s.
Validation batch 12/54. Time: 1.3s. ETA: 57s.
Validation batch 13/54. Time: 1.2s. ETA: 55s.
Validation batch 14/54. Time: 1.2s. ETA: 53s.
Validation batch 15/54. Time: 1.2s. ETA: 51s.
Validation batch 16/54. Time: 1.1s. ETA: 49s.
Validation batch 17/54. Time: 1.1s. ETA: 48s.
Validation batch 18/54. Time: 1.3s. ETA: 46s.
Validation batch 19/54. Time: 1.2s. ETA: 45s.
Validation batch 20/54. Time: 1.3s. ETA: 43s.
Validation batch 21/54. Time: 1.2s. ETA: 42s.
Validation batch 2

In [10]:
vloss, vcer, vwa = validate(model, valid_imgs, valid_lbls, 256, transform_pipeline)

Validation batch 1/36. Time: 1.4s. ETA: 50s.
Validation batch 2/36. Time: 1.5s. ETA: 49s.
Validation batch 3/36. Time: 1.3s. ETA: 46s.
Validation batch 4/36. Time: 1.3s. ETA: 44s.
Validation batch 5/36. Time: 1.2s. ETA: 41s.
Validation batch 6/36. Time: 1.3s. ETA: 40s.
Validation batch 7/36. Time: 1.3s. ETA: 38s.
Validation batch 8/36. Time: 1.3s. ETA: 37s.
Validation batch 9/36. Time: 1.2s. ETA: 35s.
Validation batch 10/36. Time: 1.3s. ETA: 34s.
Validation batch 11/36. Time: 1.2s. ETA: 32s.
Validation batch 12/36. Time: 1.3s. ETA: 31s.
Validation batch 13/36. Time: 1.3s. ETA: 30s.
Validation batch 14/36. Time: 1.3s. ETA: 28s.
Validation batch 15/36. Time: 1.3s. ETA: 27s.
Validation batch 16/36. Time: 1.2s. ETA: 26s.
Validation batch 17/36. Time: 1.1s. ETA: 24s.
Validation batch 18/36. Time: 1.2s. ETA: 23s.
Validation batch 19/36. Time: 1.3s. ETA: 21s.
Validation batch 20/36. Time: 1.2s. ETA: 20s.
Validation batch 21/36. Time: 1.2s. ETA: 19s.
Validation batch 22/36. Time: 1.2s. ETA: 17

In [11]:
train_pred = infer(model, train_imgs, 256, transform_pipeline)

Inference batch 1/54. Time: 1.8s. ETA: 1m 37s.
Inference batch 2/54. Time: 1.2s. ETA: 1m 19s.
Inference batch 3/54. Time: 1.3s. ETA: 1m 13s.
Inference batch 4/54. Time: 1.3s. ETA: 1m 10s.
Inference batch 5/54. Time: 1.3s. ETA: 1m 7s.
Inference batch 6/54. Time: 1.2s. ETA: 1m 4s.
Inference batch 7/54. Time: 1.3s. ETA: 1m 3s.
Inference batch 8/54. Time: 1.2s. ETA: 1m 1s.
Inference batch 9/54. Time: 1.2s. ETA: 59s.
Inference batch 10/54. Time: 1.2s. ETA: 57s.
Inference batch 11/54. Time: 1.2s. ETA: 55s.
Inference batch 12/54. Time: 1.2s. ETA: 54s.
Inference batch 13/54. Time: 1.2s. ETA: 52s.
Inference batch 14/54. Time: 1.2s. ETA: 51s.
Inference batch 15/54. Time: 1.4s. ETA: 50s.
Inference batch 16/54. Time: 1.4s. ETA: 49s.
Inference batch 17/54. Time: 1.3s. ETA: 47s.
Inference batch 18/54. Time: 1.4s. ETA: 46s.
Inference batch 19/54. Time: 1.4s. ETA: 45s.
Inference batch 20/54. Time: 1.4s. ETA: 44s.
Inference batch 21/54. Time: 1.4s. ETA: 43s.
Inference batch 22/54. Time: 1.3s. ETA: 42s.

In [12]:
valid_pred = infer(model, valid_imgs, 256, transform_pipeline)

Inference batch 1/36. Time: 1.2s. ETA: 42s.
Inference batch 2/36. Time: 1.3s. ETA: 41s.
Inference batch 3/36. Time: 1.2s. ETA: 40s.
Inference batch 4/36. Time: 1.2s. ETA: 39s.
Inference batch 5/36. Time: 1.3s. ETA: 38s.
Inference batch 6/36. Time: 1.3s. ETA: 37s.
Inference batch 7/36. Time: 1.2s. ETA: 36s.
Inference batch 8/36. Time: 1.2s. ETA: 34s.
Inference batch 9/36. Time: 1.2s. ETA: 33s.
Inference batch 10/36. Time: 1.2s. ETA: 32s.
Inference batch 11/36. Time: 1.2s. ETA: 30s.
Inference batch 12/36. Time: 1.2s. ETA: 29s.
Inference batch 13/36. Time: 1.4s. ETA: 28s.
Inference batch 14/36. Time: 1.4s. ETA: 27s.
Inference batch 15/36. Time: 1.4s. ETA: 26s.
Inference batch 16/36. Time: 1.4s. ETA: 25s.
Inference batch 17/36. Time: 1.3s. ETA: 24s.
Inference batch 18/36. Time: 1.3s. ETA: 23s.
Inference batch 19/36. Time: 1.2s. ETA: 21s.
Inference batch 20/36. Time: 1.2s. ETA: 20s.
Inference batch 21/36. Time: 1.2s. ETA: 19s.
Inference batch 22/36. Time: 1.1s. ETA: 17s.
Inference batch 23/

In [14]:
with open(os.path.join(model.model_path_, 'train_res.txt'), 'w') as f:
    f.write('\n'.join(['"{}" -> "{}"'.format(*x) for x in zip(train_lbls, train_pred)]))

In [15]:
with open(os.path.join(model.model_path_, 'valid_res.txt'), 'w') as f:
    f.write('\n'.join(['"{}" -> "{}"'.format(*x) for x in zip(valid_lbls, valid_pred)]))

In [9]:
%%time
fname = 'D:/Data/bujo_sample_v2/dataset.json'
extraction_path = os.path.join(os.path.dirname(fname),
                               os.path.basename(fname).split('.')[0])
with open(fname, 'r', encoding='utf-8') as f:
    src = json.load(f)
    
words = wp.extract_words_from_dataset(src, (1,))

transforms = [{'cutoff': 0.7, 'type': 'cutoff'},
  {'type': 'trimx'}, {'type': 'trimy'},
  {'type': 'resize', 'y': 32}, {'type': 'invert'}]

raw_lbls = [x[0] for x in words]

def load_images(path, words, transform):
    res = []
    for (i, (word, fname)) in enumerate(tqdm.tqdm(words)):
        src_image = skimage.io.imread(os.path.join(path, fname), as_grey=True)
        res_image = wp.perform_transform(src_image, transform)
        res.append(res_image)
    return res

Wall time: 39 ms


In [10]:
raw_imgs = load_images('D:/Data/bujo_sample_v2/dataset/', words, transforms)

  0%|                                                                                         | 0/1579 [00:00<?, ?it/s]C:\Anaconda3\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')
C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|█████████████████████████████████████████████████████████████████████████████| 1579/1579 [00:12<00:00, 131.00it/s]


In [21]:
raw_pred = infer(model, raw_imgs, 256, transform_pipeline)

Inference batch 1/6. Time: 1.3s. ETA: 6.5s.
Inference batch 2/6. Time: 1.3s. ETA: 5.2s.
Inference batch 3/6. Time: 1.2s. ETA: 3.8s.
Inference batch 4/6. Time: 1.2s. ETA: 2.5s.
Inference batch 5/6. Time: 1.2s. ETA: 1.3s.
Inference batch 6/6. Time: 1.2s. ETA: 0s.
Inference complete.


In [24]:
raw_out = inferRaw(model, raw_imgs, 256, transform_pipeline)

C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Inference batch 1/6. Time: 1.3s. ETA: 6.3s.
Inference batch 2/6. Time: 1.2s. ETA: 5.0s.
Inference batch 3/6. Time: 1.1s. ETA: 3.6s.
Inference batch 4/6. Time: 1.1s. ETA: 2.4s.
Inference batch 5/6. Time: 1.2s. ETA: 1.2s.
Inference batch 6/6. Time: 1.1s. ETA: 0s.
Inference complete.


In [25]:
raw_out.shape

(1536, 32, 102)

In [26]:
raw_out[0]

array([[-1.6308817 , -2.814611  ,  1.59136   , ..., -0.10597652,
        -2.0938945 ,  7.099291  ],
       [-4.2038293 , -3.867228  , -1.0772402 , ..., -4.207089  ,
        -2.6427054 , 14.466589  ],
       [-5.112555  , -3.6673427 , -3.209397  , ..., -3.8577573 ,
        -1.5834627 , 15.831958  ],
       ...,
       [-1.8985775 ,  0.8956639 , -1.6348583 , ...,  1.2534647 ,
        -1.5171933 , 22.063568  ],
       [-1.337069  ,  0.74955535, -1.2963966 , ...,  1.5676981 ,
        -1.266425  , 22.06962   ],
       [-0.933487  ,  0.56467247, -1.221588  , ...,  2.169692  ,
        -0.7691835 , 21.261189  ]], dtype=float32)

In [23]:
with open(os.path.join(model.model_path_, 'raw_res.txt'), 'w') as f:
    f.write('\n'.join(['"{}" -> "{}"'.format(*x) for x in zip(raw_lbls, raw_pred)]))

In [25]:
import json

add_info = dict(chars=''.join(model.chars_),
                textlen=model.text_len_,
                output={
                    'decoded': model.tf_decoder_[1].name.split(':')[0],
                    'raw': model.tf_rnn_out_.name.split(':')[0]
                },
               input={
                   'images': model.tf_in_images_.name.split(':')[0],
               })

In [27]:
with open(os.path.join(model.model_path_, 'model_info.json'), 'w') as fp:
    json.dump(add_info, fp)

In [30]:
%%time
tflite_buff = model.getTFLiteBuffer()
with open(os.path.join(model.model_path_, 'model.tflite'), "wb") as f:
    f.write(tflite_buff)

Wall time: 9.12 s


In [31]:
tflite_fname = os.path.join(model.model_path_, 'model.tflite')
tflite_fname

'D:/models/htr-static-128/model.tflite'

In [32]:
interpreter = tf.lite.Interpreter(model_path=tflite_fname)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [87]:
input_details

[{'name': 'in_image',
  'index': 24,
  'shape': array([  1, 128,  32]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [88]:
output_details

[{'name': 'out_rnn',
  'index': 25,
  'shape': array([  1,  32, 102]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [49]:
tmp = transform_pipeline.transform(raw_imgs[0]).reshape((1, 128, 32)).astype(np.float32)

C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [89]:
tflite_in = interpreter.tensor(input_details[0]['index'])
tflite_out = interpreter.tensor(output_details[0]['index'])

In [90]:
tflite_in().fill(0.0)
tflite_out().fill(0.0)

In [91]:
interpreter.invoke()
#tmp = interpreter.get_tensor(output_details[0]['index'])

RuntimeError: There is at least 1 reference to internal data
      in the interpreter in the form of a numpy array or slice. Be sure to
      only hold the function returned from tensor() if you are using raw
      data access.

<tf.Tensor 'Const_1:0' shape=(1, 128, 32) dtype=float32>